In [ ]:
"""
LangGraph Workflow 정의
게임의 전체 흐름을 그래프로 구성
"""

from langgraph.graph import StateGraph, END
from langchain_teddynote.graphs import visualize_graph
from state import GameState
from nodes import (
    setup_game_node,
    character_speak_node,
    user_input_node,
    vote_node,
    next_turn_node
)


def should_continue_discussion(state: GameState) -> str:
    """
    대화를 계속할지 결정하는 조건부 엣지
    """
    phase = state.get("phase", "discussion")

    if phase == "end":
        return "end"
    elif state.get("user_input"):
        return "user_input"
    elif state.get("user_target"):
        return "vote"
    else:
        return "character_speak"


def create_game_graph():
    """
    마피아 게임 그래프 생성
    """
    # StateGraph 초기화
    workflow = StateGraph(GameState)

    # 노드 추가
    workflow.add_node("setup", setup_game_node)
    workflow.add_node("character_speak", character_speak_node)
    workflow.add_node("user_input", user_input_node)
    workflow.add_node("vote", vote_node)
    workflow.add_node("next_turn", next_turn_node)

    # 시작점: setup
    workflow.set_entry_point("setup")

    # 엣지 정의
    workflow.add_edge("setup", "next_turn")

    # next_turn 후 조건부 분기
    workflow.add_conditional_edges(
        "next_turn",
        should_continue_discussion,
        {
            "character_speak": "character_speak",
            "user_input": "user_input",
            "vote": "vote",
            "end": END
        }
    )

    # character_speak 후 다시 next_turn
    workflow.add_edge("character_speak", "next_turn")

    # user_input 후 다시 next_turn
    workflow.add_edge("user_input", "next_turn")

    # vote 후 종료
    workflow.add_edge("vote", END)

    # 컴파일
    app = workflow.compile()

    return app


# 그래프 시각화용 (선택사항)
def visualize_graph1():
    """
    그래프 구조를 시각화 (mermaid 형식)
    """
    app = create_game_graph()

    try:
        # Mermaid 다이어그램 출력
        print(app.get_graph().draw_mermaid())
        visualize_graph(app, xray=True)
        
    except Exception as e:
        print(f"시각화 실패: {e}")
        print("그래프는 정상적으로 생성되었습니다.")


if __name__ == "__main__":
    # 그래프 시각화 테스트
    visualize_graph1()



---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	setup(setup)
	character_speak(character_speak)
	user_input(user_input)
	vote(vote)
	next_turn(next_turn)
	__end__([<p>__end__</p>]):::last
	__start__ --> setup;
	character_speak --> next_turn;
	next_turn -. &nbsp;end&nbsp; .-> __end__;
	next_turn -.-> character_speak;
	next_turn -.-> user_input;
	next_turn -.-> vote;
	setup --> next_turn;
	user_input --> next_turn;
	vote --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

